# 1141_社會資料分析 作業五
學生：劉晏成

以「2018台北大學社會系三鶯調查」（brilliant_working.sav）檔案回答以下問題：  
1. 使用迴歸模型，以教育年數（eduy）預測收入（inc）
2. 使用迴歸模型，以社交性（sociablity）預測收入
3. 控制人際網路（net）之後，社交性（sociability）和收入（inc）之間的淨相關如何？與未控制之前的零階相關差別為何？

In [1]:
from load import load_sav
import pandas as pd

df = load_sav("../data/brilliant_working.sav")

In [2]:
variable_value_labels = df.attrs['variable_value_labels']
column_names = df.attrs['column_names']
column_names_to_labels = df.attrs['column_names_to_labels']

## Q1: 使用迴歸模型，以教育年數（eduy）預測收入（inc）

In [3]:
df_1 = df[['eduy', 'inc']].copy()

檢驗遺漏值狀態

In [19]:
from statistic.missing_value import check_missing_value

check_missing_value(df_1)

,# of Missing Values
eduy,0
inc,4


發現有遺漏值，刪除資料

In [22]:
df_1_cleared = df_1.dropna()
df_1_cleared

,eduy,inc
0,16.0,25000.0
1,16.0,35000.0
2,16.0,25000.0
3,12.0,35000.0
4,12.0,5000.0
...,...,...
863,6.0,0.0
864,12.0,0.0
865,6.0,0.0
866,12.0,35000.0


In [21]:
from scipy import stats

slope, intercept, r_value, p_value, std_err = stats.linregress(df_1_cleared['eduy'], df_1_cleared['inc'])
slope

np.float64(2059.8361415029276)

In [23]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Fit model
X = sm.add_constant(df_1_cleared['eduy'])  # Adds intercept
model = sm.OLS(df_1_cleared['inc'], X).fit()

# Get complete summary (includes ANOVA table)
print(model.summary())

# Extract ANOVA table specifically
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

                            OLS Regression Results                            
Dep. Variable:                    inc   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                  0.064
Method:                 Least Squares   F-statistic:                     60.05
Date:                Sat, 08 Nov 2025   Prob (F-statistic):           2.60e-14
Time:                        15:57:06   Log-Likelihood:                -10111.
No. Observations:                 864   AIC:                         2.023e+04
Df Residuals:                     862   BIC:                         2.024e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       8388.7038   3740.422      2.243      0.0

AttributeError: 'PandasData' object has no attribute 'design_info'

## Q2: 使用迴歸模型，以社交性（sociablity）預測收入

## Q3: 控制人際網路（net）之後，社交性（sociability）和收入（inc）之間的淨相關如何？與未控制之前的零階相關差別為何？

In [9]:
df_3 = df[['net', 'sociability', 'inc']].copy()
df_3

,net,sociability,inc
0,3.0,17.0,25000.0
1,6.0,16.0,35000.0
2,6.0,16.0,25000.0
3,3.0,19.0,35000.0
4,3.0,16.0,5000.0
...,...,...,...
863,4.0,13.0,0.0
864,3.0,17.0,0.0
865,6.0,22.0,0.0
866,5.0,13.0,35000.0


考察人際網路、社交性、收入等三資料屬性，其中社交性由於alpha（Cronbach’s alpha）0.574為偏低，故將sociability 視為排序變項。  
因此採用spearman 方法做相關分析。

In [5]:
from statistic.correlation import zero_order_correlation_matrix, partial_correlation_matrix

In [6]:
zero_order_correlation_matrix(df_3, ['sociability', 'inc'], method='spearman')

,Variable,Statistic,sociability,inc
0,sociability,Correlation,1.0,0.085
1,sociability,Significance (2-tailed),.,0.013
2,sociability,df,0,859
3,inc,Correlation,0.085,1.0
4,inc,Significance (2-tailed),0.013,.
5,inc,df,859,0


In [7]:
partial_correlation_matrix(df_3, ['sociability', 'inc'], 'net', method='spearman')

,Control Variable,Variable,Statistic,sociability,inc
0,net,sociability,Correlation,1.0,0.070
1,net,sociability,Significance (2-tailed),.,0.040
2,net,sociability,df,0,858
3,net,inc,Correlation,0.070,1.0
4,net,inc,Significance (2-tailed),0.040,.
5,net,inc,df,858,0


### Answer
首先觀察零階相關與淨相關的相關係數：
1. sociability 與inc 的零階相關係數為0.085
2. sociability 與inc 的淨相關係數為0.070

接下來檢定零階相關與淨相關  
$H_0: 不存在相關$
$H_1: 存在相關$

觀察零階相關與淨相關的雙尾顯著值：
1. 零階相關顯著值為0.013
2. 淨相關顯著值為0.04

兩者都拒絕需為假設，故兩者都存在相關

然而無論是零階相關或淨相關，係數絕對值皆小於 0.1，故**皆無相關性**。